In [25]:
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
# LLM libraries
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI

### 1. Creating Prompt:

In [29]:
template_string = """ 
You are an expert researcher analyst in customer success team. \
Your job is the analyze the customer feedback and provide insights to the product team. \
Given the following customer feedback, provide the below insights: \
1. Product: Name of the product in review. \
2. Summary: Provide a summary of the feedback along with the sentiment. \
3. Positives: List out the positive feedbacks. \
4. Negatives: List out the negative feedbacks. \

Customer Feedback: \
{feedback}

"""

prompt = PromptTemplate.from_template(template_string)
print(prompt)

input_variables=['feedback'] template=' \nYou are an expert researcher analyst in customer success team. Your job is the analyze the customer feedback and provide insights to the product team. Given the following customer feedback, provide the below insights: 1. Product: Name of the product in review. 2. Summary: Provide a summary of the feedback along with the sentiment. 3. Positives: List out the positive feedbacks. 4. Negatives: List out the negative feedbacks. \nCustomer Feedback: {feedback}\n'


### 2. Creating an LLM Chain:

In [30]:
### Defining the LLM chain
llm_chain = LLMChain(
    llm=ChatOpenAI(model="gpt-4o-mini",temperature=0),
    prompt=prompt
)

llm_chain

LLMChain(prompt=PromptTemplate(input_variables=['feedback'], template=' \nYou are an expert researcher analyst in customer success team. Your job is the analyze the customer feedback and provide insights to the product team. Given the following customer feedback, provide the below insights: 1. Product: Name of the product in review. 2. Summary: Provide a summary of the feedback along with the sentiment. 3. Positives: List out the positive feedbacks. 4. Negatives: List out the negative feedbacks. \nCustomer Feedback: {feedback}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fd34333be50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fd343a333d0>, model_name='gpt-4o-mini', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy=''))

### 3. Generating output:

In [31]:
review = "The samsung a70 is amazing and the customer service is very helpful. However, the product is very expensive."
output = llm_chain.invoke({"feedback":review})

In [32]:
output

{'feedback': 'The samsung a70 is amazing and the customer service is very helpful. However, the product is very expensive.',
 'text': "1. **Product:** Samsung A70\n\n2. **Summary:** The customer feedback expresses a positive sentiment towards the Samsung A70, highlighting its amazing features and the helpfulness of the customer service. However, the customer also notes a negative aspect regarding the product's high price, indicating that they find it to be very expensive.\n\n3. **Positives:**\n   - The Samsung A70 is described as amazing.\n   - The customer service is noted to be very helpful.\n\n4. **Negatives:**\n   - The product is considered very expensive."}

In [33]:
print(output['text'])

1. **Product:** Samsung A70

2. **Summary:** The customer feedback expresses a positive sentiment towards the Samsung A70, highlighting its amazing features and the helpfulness of the customer service. However, the customer also notes a negative aspect regarding the product's high price, indicating that they find it to be very expensive.

3. **Positives:**
   - The Samsung A70 is described as amazing.
   - The customer service is noted to be very helpful.

4. **Negatives:**
   - The product is considered very expensive.


### 4. Output Parsing:

In [34]:
### Defining the output schema from the LLM        
output_schema = ResponseSchema(name="review_analysis",description="Review analysis output")
output_parser = StructuredOutputParser.from_response_schemas([output_schema])
format_instructions = output_parser.get_format_instructions()

In [36]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"review_analysis": string  // Review analysis output
}
```


In [37]:
template_string = """ 
You are an expert researcher analyst in customer success team. \
Your job is the analyze the customer feedback and provide insights to the product team. \
Given the following customer feedback, provide the below insights: \
1. Product: Name of the product in review. \
2. Summary: Provide a summary of the feedback along with the sentiment. \
3. Positives: List out the positive feedbacks. \
4. Negatives: List out the negative feedbacks. \

Customer Feedback: \
{feedback}

{format_instructions}
"""

prompt = PromptTemplate.from_template(template_string)


### Defining the LLM chain
llm_chain = LLMChain(
    llm=ChatOpenAI(model="gpt-4o-mini",temperature=0),
    prompt=prompt,
    output_parser=output_parser
)

review = "The samsung a70 is amazing and the customer service is very helpful. However, the product is very expensive."
output = llm_chain.invoke({"feedback":review,"format_instructions":format_instructions})

In [38]:
output

{'feedback': 'The samsung a70 is amazing and the customer service is very helpful. However, the product is very expensive.',
 'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"review_analysis": string  // Review analysis output\n}\n```',
 'text': {'review_analysis': {'Product': 'Samsung A70',
   'Summary': 'The customer finds the Samsung A70 to be amazing and appreciates the helpful customer service, but feels that the product is very expensive. The overall sentiment is mixed, leaning towards positive.',
   'Positives': ['The Samsung A70 is amazing.',
    'Customer service is very helpful.'],
   'Negatives': ['The product is very expensive.']}}}

In [39]:
print(output['text'])

{'review_analysis': {'Product': 'Samsung A70', 'Summary': 'The customer finds the Samsung A70 to be amazing and appreciates the helpful customer service, but feels that the product is very expensive. The overall sentiment is mixed, leaning towards positive.', 'Positives': ['The Samsung A70 is amazing.', 'Customer service is very helpful.'], 'Negatives': ['The product is very expensive.']}}


In [40]:
print(output['text']['review_analysis'])

{'Product': 'Samsung A70', 'Summary': 'The customer finds the Samsung A70 to be amazing and appreciates the helpful customer service, but feels that the product is very expensive. The overall sentiment is mixed, leaning towards positive.', 'Positives': ['The Samsung A70 is amazing.', 'Customer service is very helpful.'], 'Negatives': ['The product is very expensive.']}


In [41]:
parsed_output = output['text']['review_analysis']

In [42]:
parsed_output

{'Product': 'Samsung A70',
 'Summary': 'The customer finds the Samsung A70 to be amazing and appreciates the helpful customer service, but feels that the product is very expensive. The overall sentiment is mixed, leaning towards positive.',
 'Positives': ['The Samsung A70 is amazing.',
  'Customer service is very helpful.'],
 'Negatives': ['The product is very expensive.']}

In [43]:
type(parsed_output)

dict